In [39]:
import numpy 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense

import ipywidgets as wg
from IPython.display import display
from ipywidgets import Label, Layout, HBox
import os

In [40]:
# ชื่อไฟล์
nameF = 'a15'

# ไฟล์
cols_to_use = ['Character', 'Time']
key = pd.read_csv(f"2 keys/{nameF}.csv", usecols = cols_to_use)

# ตัด + เรียงข้อมูล 
key = key[key.Time >= 0]
key = key[key.Time <= 10000]
key = key.sort_values(by=['Character'])
key

,Character,Time
275,AA,2071
2329,AA,2317
2311,AA,2836
2310,AA,2944
2272,AA,6243
...,...,...
3620,NaN,2552
3869,NaN,2121
4616,NaN,1925
6705,NaN,3674


In [41]:
cha = key.Character.unique().tolist()

# เก็บ test ,train
ltest = []
ltrain = []
lCha = []

for i in cha:
    nameC = i
    lc = key[key.Character == i]

    dataframe = pd.DataFrame(lc.Time)

    # load the dataset
    dataset = dataframe.values
    dataset = dataset.astype('float32')
    
    print(f'Character {i}')
    
#     --------------------------------------------------------------------------------------------------------------

    # split into train and test sets
    train_size = int(len(dataset) * 0.6)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    print(len(train), len(test))

    # convert an array of values into a dataset matrix
    def create_dataset(dataset, look_back=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back), 0]
            dataX.append(a)
            dataY.append(dataset[i + look_back, 0])
        return numpy.array(dataX), numpy.array(dataY)

    # reshape into X=t and Y=t+1
    look_back = 1
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)

    # create and fit Multilayer Perceptron model
    model = Sequential()
    model.add(Dense(8, input_dim=look_back, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, epochs=200, batch_size=2, verbose=2)

    # Estimate model performance
    trainScore = model.evaluate(trainX, trainY, verbose=0)
    print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, math.sqrt(trainScore)))
    testScore = model.evaluate(testX, testY, verbose=0)
    print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, math.sqrt(testScore)))

    # generate predictions for training
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    # shift train predictions for plotting
    trainPredictPlot = numpy.empty_like(dataset)
    trainPredictPlot[:, :] = numpy.nan
    trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
    # shift test predictions for plotting
    testPredictPlot = numpy.empty_like(dataset)
    testPredictPlot[:, :] = numpy.nan
    testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict


#     --------------------------------------------------------------------------------------------------------------

    # หาตัวเลขตัวสุดท้ายใน test
    for i in testPredictPlot:
        if ~np.isnan(i[0]):
            numtest = i[0]

    # หาตัวเลขตัวสุดท้ายใน train
    for i in trainPredictPlot:
        if ~np.isnan(i[0]):
            numtrain = i[0]

    # เก็บค่า test ,train
    ltrain.append(numtrain)
    ltest.append(numtest)
    lCha.append(nameC)

# ตารางไว้บันทึกค่าลงในไฟล์
test_n_train = pd.DataFrame()
test_n_train['Character'] = lCha
test_n_train['test'] = ltest
test_n_train['train'] = ltrain

# เซฟไฟล์ test ,train
test_n_train.to_csv(f"E:/dataSic/TypingTime/2 key_test_n_train/test_n_train_{nameF}.csv", index=False)
print('Saved successfully')


Character AA
141 95
Epoch 1/200
70/70 - 0s - loss: 1965549.3750
Epoch 2/200
70/70 - 0s - loss: 1717400.0000
Epoch 3/200
70/70 - 0s - loss: 1652925.8750
Epoch 4/200
70/70 - 0s - loss: 1645375.6250
Epoch 5/200
70/70 - 0s - loss: 1644814.1250
Epoch 6/200
70/70 - 0s - loss: 1644037.1250
Epoch 7/200
70/70 - 0s - loss: 1649128.6250
Epoch 8/200
70/70 - 0s - loss: 1648083.6250
Epoch 9/200
70/70 - 0s - loss: 1649413.5000
Epoch 10/200
70/70 - 0s - loss: 1641387.1250
Epoch 11/200
70/70 - 0s - loss: 1638878.0000
Epoch 12/200
70/70 - 0s - loss: 1650550.5000
Epoch 13/200
70/70 - 0s - loss: 1652559.2500
Epoch 14/200
70/70 - 0s - loss: 1647957.7500
Epoch 15/200
70/70 - 0s - loss: 1640790.3750
Epoch 16/200
70/70 - 0s - loss: 1640407.3750
Epoch 17/200
70/70 - 0s - loss: 1639136.6250
Epoch 18/200
70/70 - 0s - loss: 1643263.2500
Epoch 19/200
70/70 - 0s - loss: 1638314.3750
Epoch 20/200
70/70 - 0s - loss: 1645705.2500
Epoch 21/200
70/70 - 0s - loss: 1642450.5000
Epoch 22/200
70/70 - 0s - loss: 1640157.1250

70/70 - 0s - loss: 1630192.6250
Epoch 182/200
70/70 - 0s - loss: 1632656.2500
Epoch 183/200
70/70 - 0s - loss: 1647841.6250
Epoch 184/200
70/70 - 0s - loss: 1633390.3750
Epoch 185/200
70/70 - 0s - loss: 1648329.0000
Epoch 186/200
70/70 - 0s - loss: 1639954.8750
Epoch 187/200
70/70 - 0s - loss: 1634263.8750
Epoch 188/200
70/70 - 0s - loss: 1638594.8750
Epoch 189/200
70/70 - 0s - loss: 1633786.5000
Epoch 190/200
70/70 - 0s - loss: 1632821.0000
Epoch 191/200
70/70 - 0s - loss: 1636644.2500
Epoch 192/200
70/70 - 0s - loss: 1635571.8750
Epoch 193/200
70/70 - 0s - loss: 1637749.5000
Epoch 194/200
70/70 - 0s - loss: 1634602.3750
Epoch 195/200
70/70 - 0s - loss: 1630055.8750
Epoch 196/200
70/70 - 0s - loss: 1638854.1250
Epoch 197/200
70/70 - 0s - loss: 1638919.0000
Epoch 198/200
70/70 - 0s - loss: 1633725.3750
Epoch 199/200
70/70 - 0s - loss: 1635951.1250
Epoch 200/200
70/70 - 0s - loss: 1632532.7500
Train Score: 1626589.00 MSE (1275.38 RMSE)
Test Score: 1963340.75 MSE (1401.19 RMSE)
Character

Epoch 133/200
2/2 - 0s - loss: 2618486.7500
Epoch 134/200
2/2 - 0s - loss: 2598126.5000
Epoch 135/200
2/2 - 0s - loss: 2581220.5000
Epoch 136/200
2/2 - 0s - loss: 2574419.7500
Epoch 137/200
2/2 - 0s - loss: 2558431.0000
Epoch 138/200
2/2 - 0s - loss: 2540629.0000
Epoch 139/200
2/2 - 0s - loss: 2522097.2500
Epoch 140/200
2/2 - 0s - loss: 2509389.0000
Epoch 141/200
2/2 - 0s - loss: 2494639.2500
Epoch 142/200
2/2 - 0s - loss: 2479714.0000
Epoch 143/200
2/2 - 0s - loss: 2472338.5000
Epoch 144/200
2/2 - 0s - loss: 2458635.2500
Epoch 145/200
2/2 - 0s - loss: 2441556.0000
Epoch 146/200
2/2 - 0s - loss: 2427652.7500
Epoch 147/200
2/2 - 0s - loss: 2423596.2500
Epoch 148/200
2/2 - 0s - loss: 2406881.7500
Epoch 149/200
2/2 - 0s - loss: 2396081.2500
Epoch 150/200
2/2 - 0s - loss: 2378977.5000
Epoch 151/200
2/2 - 0s - loss: 2375841.5000
Epoch 152/200
2/2 - 0s - loss: 2357938.0000
Epoch 153/200
2/2 - 0s - loss: 2347379.7500
Epoch 154/200
2/2 - 0s - loss: 2342950.5000
Epoch 155/200
2/2 - 0s - loss: 2

Epoch 122/200
3/3 - 0s - loss: 313103.8438
Epoch 123/200
3/3 - 0s - loss: 313982.3125
Epoch 124/200
3/3 - 0s - loss: 312863.6875
Epoch 125/200
3/3 - 0s - loss: 313713.5000
Epoch 126/200
3/3 - 0s - loss: 312697.8750
Epoch 127/200
3/3 - 0s - loss: 313314.5625
Epoch 128/200
3/3 - 0s - loss: 313313.0938
Epoch 129/200
3/3 - 0s - loss: 313602.4375
Epoch 130/200
3/3 - 0s - loss: 313122.6875
Epoch 131/200
3/3 - 0s - loss: 313307.9688
Epoch 132/200
3/3 - 0s - loss: 313189.5312
Epoch 133/200
3/3 - 0s - loss: 314621.8125
Epoch 134/200
3/3 - 0s - loss: 312750.4375
Epoch 135/200
3/3 - 0s - loss: 313477.0000
Epoch 136/200
3/3 - 0s - loss: 312875.0938
Epoch 137/200
3/3 - 0s - loss: 313641.9375
Epoch 138/200
3/3 - 0s - loss: 313935.8125
Epoch 139/200
3/3 - 0s - loss: 313452.5938
Epoch 140/200
3/3 - 0s - loss: 312727.5625
Epoch 141/200
3/3 - 0s - loss: 312291.9375
Epoch 142/200
3/3 - 0s - loss: 313353.1562
Epoch 143/200
3/3 - 0s - loss: 312960.2500
Epoch 144/200
3/3 - 0s - loss: 312723.6562
Epoch 145/2

Epoch 111/200
2/2 - 0s - loss: 2374406.0000
Epoch 112/200
2/2 - 0s - loss: 2333410.0000
Epoch 113/200
2/2 - 0s - loss: 2302494.0000
Epoch 114/200
2/2 - 0s - loss: 2267988.5000
Epoch 115/200
2/2 - 0s - loss: 2233769.0000
Epoch 116/200
2/2 - 0s - loss: 2201510.0000
Epoch 117/200
2/2 - 0s - loss: 2161641.0000
Epoch 118/200
2/2 - 0s - loss: 2131295.0000
Epoch 119/200
2/2 - 0s - loss: 2099067.0000
Epoch 120/200
2/2 - 0s - loss: 2066238.7500
Epoch 121/200
2/2 - 0s - loss: 2039561.7500
Epoch 122/200
2/2 - 0s - loss: 2008020.2500
Epoch 123/200
2/2 - 0s - loss: 1978793.6250
Epoch 124/200
2/2 - 0s - loss: 1947899.6250
Epoch 125/200
2/2 - 0s - loss: 1911721.1250
Epoch 126/200
2/2 - 0s - loss: 1884090.8750
Epoch 127/200
2/2 - 0s - loss: 1858018.2500
Epoch 128/200
2/2 - 0s - loss: 1826933.0000
Epoch 129/200
2/2 - 0s - loss: 1798840.7500
Epoch 130/200
2/2 - 0s - loss: 1769399.1250
Epoch 131/200
2/2 - 0s - loss: 1745880.5000
Epoch 132/200
2/2 - 0s - loss: 1716569.0000
Epoch 133/200
2/2 - 0s - loss: 1

UnboundLocalError: local variable 'logs' referenced before assignment

# - - - -  - - - -  - - - -  - - - -  - - - -  - - - -  - - - -  - - - -  - - - -  - - - -  - - - -  - - - -  - - - -  - - - 

In [ ]:
# ไฟล์นำมาคำนวนค่าเฉลี่ย,thrsehold
cols_to_use = ['Character', 'test' ,'train']
Tthreshold = pd.read_csv(f'2 key_test_n_train/test_n_train_a13.csv' ,usecols = cols_to_use)
Tthreshold

In [ ]:
# รับข้อมูลมาเทียบ
testcompare  = pd.read_csv(f"2 keys/a13.csv",usecols = ["Character" ,"Time"])

# ตัด + เรียงข้อมูลเทียบ
testcompare  = testcompare[testcompare.Time >= 0]
testcompare  = testcompare[testcompare.Time <= 10000]
testcompare  = testcompare.sort_values(by=['Character'])

# สุ่มตัวอย่างของเทียบ
testcompare = testcompare.groupby("Character").sample(n=1,replace=True)
testcompare

In [ ]:
# ตัด Character
testcompare = testcompare.loc[testcompare['Character'].isin(Tthreshold['Character'])]
testcompare

## Test

In [ ]:
# threshold Test
Test = pd.DataFrame()
Test['Character'] = Tthreshold['Character']
Test['TestPredictPlot'] = Tthreshold['test']
Test['Per5'] = Tthreshold['test'] * 0.05
Test['Per10'] = Tthreshold['test'] * 0.1
Test['Per15'] = Tthreshold['test'] * 0.15
Test['Per20'] = Tthreshold['test'] * 0.2

Test['T_Per5'] = Tthreshold['test'] + Test['Per5']
Test['B_Per5'] = Tthreshold['test'] - Test['Per5']

Test['T_Per10'] = Tthreshold['test'] + Test['Per10']
Test['B_Per10'] = Tthreshold['test'] - Test['Per10']

Test['T_Per15'] = Tthreshold['test'] + Test['Per15']
Test['B_Per15'] = Tthreshold['test'] - Test['Per15']

Test['T_Per20'] = Tthreshold['test'] + Test['Per20']
Test['B_Per20'] = Tthreshold['test'] - Test['Per20']

Test

In [ ]:
check_userTest = pd.DataFrame()

# เก็บค่าที่อยู่นอกหรือใน teshold
checker = []
check_userTest['Character'] = testcompare['Character'] 
    
# แปลงเป็น dic
data = dict(zip(testcompare['Character'],testcompare['Time']))

t5 = dict(zip(Test.Character,Test['T_Per5']))
b5 = dict(zip(Test.Character,Test['B_Per5']))

t10 = dict(zip(Test.Character,Test['T_Per10']))
b10 = dict(zip(Test.Character,Test['B_Per10']))

t15 = dict(zip(Test.Character,Test['T_Per15']))
b15 = dict(zip(Test.Character,Test['B_Per15']))

t20 = dict(zip(Test.Character,Test['T_Per20']))
b20 = dict(zip(Test.Character,Test['B_Per20']))

# ฟังก์ชันวนเช็คค่า
def check5():
    for k,v in data.items():
        if v > t5[k[:2]] or v < b5[k[:2]]:
            checker.append(0)
        else :
            checker.append(1)
            
def check10():
    for k,v in data.items():
        if v > t10[k[:2]] or v < b10[k[:2]]:
            checker.append(0)
        else :
            checker.append(1)

def check15():
    for k,v in data.items():
        if v > t15[k[:2]] or v < b15[k[:2]]:
            checker.append(0)
        else :
            checker.append(1)
            
def check20():
    for k,v in data.items():
        if v > t20[k[:2]] or v < b20[k[:2]]:
            checker.append(0)
        else :
            checker.append(1)

# เรียกฟังก์ชัน
check20()
check_userTest['checker'] = checker

check_userTest

In [ ]:
# นับจำนวนที่อยู่ในและนอกteshold
# 1 = in >> real user
# 0 = out >> fake user
num = check_userTest['checker'].to_list()
num1 = 0
num0 = 0
for i in num:
    if i== 1:
        num1 += 1
    elif i == 0:
        num0 += 1

user = num1 / (num1+num0)*100

if user > 50:
    result = "real user"
elif user == 50:
    result = "unidentify"
elif user < 50:
    result = "fake user"


print("true = ",num1)
print("false = ",num0)    
print(f'{user:.2f}% = {result}')


In [ ]:
# ฟังก์ชันไว้คำนวน %

def pec_check(functionCheck):
    checker2 = []
    
    # ฟังก์ชันวนเช็คค่า
    def check5():
        for k,v in data.items():
            if v > t5[k[:2]] or v < b5[k[:2]]:
                checker2.append(0)
            else :
                checker2.append(1)

    def check10():
        for k,v in data.items():
            if v > t10[k[:2]] or v < b10[k[:2]]:
                checker2.append(0)
            else :
                checker2.append(1)

    def check15():
        for k,v in data.items():
            if v > t15[k[:2]] or v < b15[k[:2]]:
                checker2.append(0)
            else :
                checker2.append(1)

    def check20():
        for k,v in data.items():
            if v > t20[k[:2]] or v < b20[k[:2]]:
                checker2.append(0)
            else :
                checker2.append(1)

    # เรียกฟังก์ชัน
    eval(functionCheck)

    
    num = checker2
    num1 = 0
    num0 = 0
    for i in num:
        if i== 1:
            num1 += 1
        elif i == 0:
            num0 += 1

    user = num1 / (num1+num0)*100

    if user > 50:
        result = "real user"
    elif user == 50:
        result = "unidentify"
    elif user < 50:
        result = "fake user"


    print("true = ",num1)
    print("false = ",num0)    
    print(f'{user:.2f}% = {result}')

In [ ]:
output1 = wg.Output()
output2 = wg.Output()
output3 = wg.Output()
output4 = wg.Output()
display(HBox([output1,output2,output3,output4]))

def thresholdTest(Per5 = False ,Per10 = False ,Per15 = False ,Per20 = True):
    fig, ax = plt.subplots(figsize=(10,5))
    plt.scatter(Test['Character'],Test['TestPredictPlot'])
    plt.scatter(testcompare['Character'],testcompare['Time'],marker='+')
    
    plt.title(f"1key RNN Test")
    plt.xlabel("Character")
    plt.ylabel("Time : ms")
    
    output2.clear_output()
    output1.clear_output()
    output3.clear_output()
    output4.clear_output()
    
    if Per5 == True:
        plt.plot(Test['Character'],Test['T_Per5'],color="#E00296")
        plt.plot(Test['Character'],Test['B_Per5'],color="#E00296")
        with output1:
            print('threshold 5%')
            pec_check("check5()")
            print("--------------------")
        
    if Per10 == True:
        plt.plot(Test['Character'],Test['T_Per10'],color="red")
        plt.plot(Test['Character'],Test['B_Per10'],color="red")
        with output2:
            print('threshold 10%')
            pec_check("check10()")
            print("--------------------")
        
    if Per15 == True:
        plt.plot(Test['Character'],Test['T_Per15'],color="orange")
        plt.plot(Test['Character'],Test['B_Per15'],color="orange")
        with output3:
            print('threshold 15%')
            pec_check("check15()")
            print("--------------------")
        
        
    if Per20 == True:
        plt.plot(Test['Character'],Test['T_Per20'],color="green")
        plt.plot(Test['Character'],Test['B_Per20'],color="green")
        with output4:
            print('threshold 20%')
            pec_check("check20()")
            print("--------------------")
        
    plt.show()

wg.interact(thresholdTest ,Per5 = False ,Per10 = False ,Per15 = False ,Per20 = True)

## Train

In [ ]:
# threshold Train
Train = pd.DataFrame()
Train['Character'] = Tthreshold['Character']
Train['TrainPredictPlot'] = Tthreshold['train']
Train['Per5'] = Tthreshold['train'] * 0.05
Train['Per10'] = Tthreshold['train'] * 0.1
Train['Per15'] = Tthreshold['train'] * 0.15
Train['Per20'] = Tthreshold['train'] * 0.2

Train['T_Per5'] = Tthreshold['train'] + Train['Per5']
Train['B_Per5'] = Tthreshold['train'] - Train['Per5']

Train['T_Per10'] = Tthreshold['train'] + Train['Per10']
Train['B_Per10'] = Tthreshold['train'] - Train['Per10']

Train['T_Per15'] = Tthreshold['train'] + Train['Per15']
Train['B_Per15'] = Tthreshold['train'] - Train['Per15']

Train['T_Per20'] = Tthreshold['train'] + Train['Per20']
Train['B_Per20'] = Tthreshold['train'] - Train['Per20']

Train

In [ ]:
check_userTrain = pd.DataFrame()

# เก็บค่าที่อยู่นอกหรือใน teshold
checker = []
check_userTrain['Character'] = testcompare['Character'] 
    
# แปลงเป็น dic
data = dict(zip(testcompare['Character'],testcompare['Time']))

t5 = dict(zip(Train.Character,Train['T_Per5']))
b5 = dict(zip(Train.Character,Train['B_Per5']))

t10 = dict(zip(Train.Character,Train['T_Per10']))
b10 = dict(zip(Train.Character,Train['B_Per10']))

t15 = dict(zip(Train.Character,Train['T_Per15']))
b15 = dict(zip(Train.Character,Train['B_Per15']))

t20 = dict(zip(Train.Character,Train['T_Per20']))
b20 = dict(zip(Train.Character,Train['B_Per20']))

# ฟังก์ชันวนเช็คค่า
def check5():
    for k,v in data.items():
        if v > t5[k[:2]] or v < b5[k[:2]]:
            checker.append(0)
        else :
            checker.append(1)
            
def check10():
    for k,v in data.items():
        if v > t10[k[:2]] or v < b10[k[:2]]:
            checker.append(0)
        else :
            checker.append(1)

def check15():
    for k,v in data.items():
        if v > t15[k[:2]] or v < b15[k[:2]]:
            checker.append(0)
        else :
            checker.append(1)
            
def check20():
    for k,v in data.items():
        if v > t20[k[:2]] or v < b20[k[:2]]:
            checker.append(0)
        else :
            checker.append(1)

# เรียกฟังก์ชัน
check20()
check_userTrain['checker'] = checker

check_userTrain

In [ ]:
# นับจำนวนที่อยู่ในและนอกteshold
# 1 = in >> real user
# 0 = out >> fake user
num = check_userTrain['checker'].to_list()
num1 = 0
num0 = 0
for i in num:
    if i== 1:
        num1 += 1
    elif i == 0:
        num0 += 1

user = num1 / (num1+num0)*100

if user > 50:
    result = "real user"
elif user == 50:
    result = "unidentify"
elif user < 50:
    result = "fake user"


print("true = ",num1)
print("false = ",num0)    
print(f'{user:.2f}% = {result}')


In [ ]:
# ฟังก์ชันไว้คำนวน %

def pec_check(functionCheck):
    checker2 = []
    
    # ฟังก์ชันวนเช็คค่า
    def check5():
        for k,v in data.items():
            if v > t5[k[:2]] or v < b5[k[:2]]:
                checker2.append(0)
            else :
                checker2.append(1)

    def check10():
        for k,v in data.items():
            if v > t10[k[:2]] or v < b10[k[:2]]:
                checker2.append(0)
            else :
                checker2.append(1)

    def check15():
        for k,v in data.items():
            if v > t15[k[:2]] or v < b15[k[:2]]:
                checker2.append(0)
            else :
                checker2.append(1)

    def check20():
        for k,v in data.items():
            if v > t20[k[:2]] or v < b20[k[:2]]:
                checker2.append(0)
            else :
                checker2.append(1)

    # เรียกฟังก์ชัน
    eval(functionCheck)

    
    num = checker2
    num1 = 0
    num0 = 0
    for i in num:
        if i== 1:
            num1 += 1
        elif i == 0:
            num0 += 1

    user = num1 / (num1+num0)*100

    if user > 50:
        result = "real user"
    elif user == 50:
        result = "unidentify"
    elif user < 50:
        result = "fake user"


    print("true = ",num1)
    print("false = ",num0)    
    print(f'{user:.2f}% = {result}')

In [ ]:
output1 = wg.Output()
output2 = wg.Output()
output3 = wg.Output()
output4 = wg.Output()
display(HBox([output1,output2,output3,output4]))

def thresholdTrain(Per5 = False ,Per10 = False ,Per15 = False ,Per20 = True):
    fig, ax = plt.subplots(figsize=(10,5))
    plt.scatter(Train['Character'],Train['TrainPredictPlot'])
    plt.scatter(testcompare['Character'],testcompare['Time'],marker='+')
    
    plt.title(f"1key RNN Train")
    plt.xlabel("Character")
    plt.ylabel("Time : ms")
    
    output2.clear_output()
    output1.clear_output()
    output3.clear_output()
    output4.clear_output()
    
    if Per5 == True:
        plt.plot(Train['Character'],Train['T_Per5'],color="#E00296")
        plt.plot(Train['Character'],Train['B_Per5'],color="#E00296")
        with output1:
            print('threshold 5%')
            pec_check("check5()")
            print("--------------------")
        
    if Per10 == True:
        plt.plot(Train['Character'],Train['T_Per10'],color="red")
        plt.plot(Train['Character'],Train['B_Per10'],color="red")
        with output2:
            print('threshold 10%')
            pec_check("check10()")
            print("--------------------")
        
    if Per15 == True:
        plt.plot(Train['Character'],Train['T_Per15'],color="orange")
        plt.plot(Train['Character'],Train['B_Per15'],color="orange")
        with output3:
            print('threshold 15%')
            pec_check("check15()")
            print("--------------------")
        
        
    if Per20 == True:
        plt.plot(Train['Character'],Train['T_Per20'],color="green")
        plt.plot(Train['Character'],Train['B_Per20'],color="green")
        with output4:
            print('threshold 20%')
            pec_check("check20()")
            print("--------------------")
        
    plt.show()

wg.interact(thresholdTrain ,Per5 = False ,Per10 = False ,Per15 = False ,Per20 = True)